# KDD Cup 1999 Data(標準化、次元圧縮、SVM)

http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html

In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyplot
from sklearn import datasets
from numpy import logspace
import sklearn.preprocessing as sp
from sklearn.externals import joblib
from sklearn.datasets import load_digits
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.externals import joblib
import sklearn
from sklearn.decomposition import PCA
import time
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
% matplotlib inline

In [42]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.18.1.



|ファイル名|ファイル内容|
|---|---|
|kddcup.data|フルデータ|
|kddcup.data_10_percent|フルデータの10%を抽出した学習用データ|
|corrected|正常・攻撃のラベル付けがなされた評価用データ|
|kddcup.testdata.unlabeled|正常・攻撃のラベル付けがなされていないデータ|
|kddcup.testdata.unlabeled_10_percent|正常・攻撃のラベル付けがなされていないデータの10%サブセット|
|kddcup.newtestdata_10_percent_unlabeled|正常・攻撃のラベル付けがなされていないデータの10%サブセット|

In [43]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
             "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
             "logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations",
             "num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count",
             "srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
             "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
             "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
             "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
             "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

In [44]:
kdd_data_10percent = joblib.load('dump/kddcup.data/kdd_data_10percent.00.pkl')


In [45]:
kdd_data_10percent.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,1,24,9,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.
1,0,1,24,9,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
2,0,1,24,9,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,1,24,9,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
4,0,1,24,9,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


In [46]:
kdd_data_10percent.columns.size

42

In [47]:
kdd_data_10percent.label.value_counts().size

23

In [63]:
kdd_data_10percent.shape

(494021, 42)

In [74]:
tmp = resample(kdd_data_10percent,n_samples=2000,random_state=0)

In [75]:
tmp.shape

(2000, 42)

In [76]:
X = tmp.drop(['label','service'],axis=1)

In [77]:
Y = tmp['label'].copy()

In [78]:
Y.value_counts()

smurf.          1111
neptune.         430
normal.          422
ipsweep.           9
back.              9
teardrop.          5
satan.             5
warezclient.       4
portsweep.         3
nmap.              2
Name: label, dtype: int64

In [79]:
(x_train, x_test,y_train, y_test) = train_test_split(X, Y, test_size=0.3, random_state=0,)

In [80]:
pipeline = Pipeline([
        ('standard_scaler', StandardScaler()),
    ('pca', PCA()),
        ('svm', SVC())])

In [81]:
score = 'f1'
params = {
    "pca__n_components" : list(range(2, 20)),
    'svm__kernel' : ["linear", "poly", "rbf", "sigmoid"],
    'svm__C' : np.logspace(0, 2, 5),
    'svm__gamma' : np.logspace(-3, 0, 5)
}

In [82]:
t1=time.perf_counter()
# Grid Searchを行う
clf = GridSearchCV(pipeline, params# 最適化したいパラメータセット 
    ,cv=5, # 交差検定の回数
    scoring='%s_weighted' % score ) # モデルの評価関数の指定
t2=time.perf_counter()

In [83]:
print(t2-t1,"秒")

0.00012038997374475002 秒


In [84]:
x_train.shape

(1400, 40)

In [85]:
t1=time.perf_counter()
clf.fit(x_train, y_train)
t2=time.perf_counter()

/Users/mhaya/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)
/Users/mhaya/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mhaya/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mhaya/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being 

In [86]:
print(t2-t1,"秒")

195.25765823601978 秒


In [87]:
clf.grid_scores_

[mean: 0.98657, std: 0.00766, params: {'pca__n_components': 2, 'svm__C': 1.0, 'svm__gamma': 0.001, 'svm__kernel': 'linear'},
 mean: 0.39536, std: 0.00707, params: {'pca__n_components': 2, 'svm__C': 1.0, 'svm__gamma': 0.001, 'svm__kernel': 'poly'},
 mean: 0.96100, std: 0.01221, params: {'pca__n_components': 2, 'svm__C': 1.0, 'svm__gamma': 0.001, 'svm__kernel': 'rbf'},
 mean: 0.96079, std: 0.01689, params: {'pca__n_components': 2, 'svm__C': 1.0, 'svm__gamma': 0.001, 'svm__kernel': 'sigmoid'},
 mean: 0.98657, std: 0.00766, params: {'pca__n_components': 2, 'svm__C': 1.0, 'svm__gamma': 0.005623413251903491, 'svm__kernel': 'linear'},
 mean: 0.68079, std: 0.00987, params: {'pca__n_components': 2, 'svm__C': 1.0, 'svm__gamma': 0.005623413251903491, 'svm__kernel': 'poly'},
 mean: 0.97454, std: 0.01484, params: {'pca__n_components': 2, 'svm__C': 1.0, 'svm__gamma': 0.005623413251903491, 'svm__kernel': 'rbf'},
 mean: 0.96885, std: 0.01343, params: {'pca__n_components': 2, 'svm__C': 1.0, 'svm__gamma

In [88]:
print('Best_estimator = {0}'.format(clf.best_estimator_.get_params()))

Best_estimator = {'steps': [('standard_scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=7, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('svm', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.005623413251903491,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))], 'standard_scaler': StandardScaler(copy=True, with_mean=True, with_std=True), 'pca': PCA(copy=True, iterated_power='auto', n_components=7, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False), 'svm': SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.005623413251903491,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False), 'standard_scaler__copy': True, 'standard_scaler__with_mean': True, 's

In [89]:
clf.best_estimator_

Pipeline(steps=[('standard_scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=7, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('svm', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.005623413251903491,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))])

In [90]:
t1=time.perf_counter()
pred = clf.predict(x_test)
t2=time.perf_counter()

In [91]:
print(t2-t1,"秒")

0.0031401769956573844 秒


In [92]:
print(classification_report(y_test, pred))
print(confusion_matrix(y_test, pred))

              precision    recall  f1-score   support

       back.       1.00      0.83      0.91         6
    ipsweep.       1.00      1.00      1.00         1
    neptune.       0.98      1.00      0.99       119
     normal.       0.97      1.00      0.98       131
  portsweep.       0.00      0.00      0.00         2
      satan.       1.00      1.00      1.00         1
      smurf.       1.00      1.00      1.00       335
   teardrop.       1.00      0.33      0.50         3
warezclient.       0.00      0.00      0.00         2

 avg / total       0.98      0.99      0.98       600

[[  5   0   0   1   0   0   0   0   0]
 [  0   1   0   0   0   0   0   0   0]
 [  0   0 119   0   0   0   0   0   0]
 [  0   0   0 131   0   0   0   0   0]
 [  0   0   2   0   0   0   0   0   0]
 [  0   0   0   0   0   1   0   0   0]
 [  0   0   0   0   0   0 335   0   0]
 [  0   0   1   1   0   0   0   1   0]
 [  0   0   0   2   0   0   0   0   0]]


/Users/mhaya/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
